# Imports

In [1]:
import math
import os
import torch
import torch.distributions.constraints as constraints
import pyro
import numpy as np
import copy
import time
import snakeviz
from pyro.optim import Adam
from pyro.infer import SVI, Trace_ELBO
import pyro.distributions as dist
import random
from sklearn.datasets import load_diabetes
assert pyro.__version__.startswith('1.8.4')

# clear the param store in case we're in a REPL
pyro.clear_param_store()

# Load Datasets

### Random Dataset

In [2]:
def generate_stdp_dataset(dim, num_examples, min_value, max_value):
    X = np.random.random((num_examples + 1, dim)) * (max_value - min_value) + min_value
    beta = np.random.random((dim)) * (max_value - min_value) + min_value

    noise = np.random.normal(0, np.sqrt(max_value - min_value), num_examples + 1)
    Y = X[:num_examples + 1] @ beta + noise

    X = np.asfortranarray(X)
    Y = np.asfortranarray(Y)
    X /= np.linalg.norm(X, axis=0)
    Y = (Y - Y.mean()) / Y.std()
    Y = Y * max_value

    Y = Y/np.linalg.norm(Y)

    return X, Y, beta

In [3]:
X, Y, beta = generate_stdp_dataset(3, 10, 0, 1)

### Diabetes Dataset

In [4]:
X, Y = load_diabetes(return_X_y = True)

In [5]:
print(Y[0:3])
Y = (Y - Y.mean()) / Y.std()
print(Y[0:3])
X = X / np.linalg.norm(X)
print(X[0:3])

[151.  75. 141.]
[-0.01471948 -1.00165882 -0.14457991]
[[ 0.01204066  0.01602646  0.01951005  0.00691665 -0.0139847  -0.01101129
  -0.01372455 -0.00081975  0.0062956  -0.00558019]
 [-0.00059515 -0.01411692 -0.01627753 -0.00832559 -0.00267172 -0.00605998
   0.023531   -0.0124889  -0.02160776 -0.02915748]
 [ 0.02697388  0.01602646  0.01405671 -0.0017932  -0.01441981 -0.01081324
  -0.01023184 -0.00081975  0.0009056  -0.00819989]]


In [6]:
print(X)
print(Y)

[[ 0.01204066  0.01602646  0.01951005 ... -0.00081975  0.0062956
  -0.00558019]
 [-0.00059515 -0.01411692 -0.01627753 ... -0.0124889  -0.02160776
  -0.02915748]
 [ 0.02697388  0.01602646  0.01405671 ... -0.00081975  0.0009056
  -0.00819989]
 ...
 [ 0.01318937  0.01602646 -0.00503    ... -0.00350365 -0.01482459
   0.0048986 ]
 [-0.01437966 -0.01411692  0.01235254 ...  0.00839889  0.01408111
  -0.00819989]
 [-0.01437966 -0.01411692 -0.02309421 ... -0.0124889  -0.00133444
   0.00096905]]
[-1.47194752e-02 -1.00165882e+00 -1.44579915e-01  6.99512942e-01
 -2.22496178e-01 -7.15965848e-01 -1.83538046e-01 -1.15749134e+00
 -5.47147277e-01  2.05006151e+00 -6.64021672e-01 -1.07957508e+00
  3.48889755e-01  4.26806019e-01 -4.43258925e-01  2.45001404e-01
  1.80071184e-01 -1.05621783e-01 -7.15965848e-01  2.06043272e-01
 -1.09256112e+00 -1.33929596e+00 -1.09256112e+00  1.20596866e+00
  4.13819975e-01  6.47568766e-01 -1.96524090e-01 -8.71798376e-01
 -2.74440354e-01  1.69943833e+00 -3.00412442e-01 -1.209

In [7]:
print(len(X[0]))

10


In [8]:
# Clip the dataset
# X = X[0:100]
# Y = Y[0:100]

### Set up X, Y train

In [9]:
X_train = copy.deepcopy(X)
Y_train = copy.deepcopy(Y[:len(Y) - 1])
X_train = [torch.tensor(member) for member in X_train]
Y_train = [torch.tensor(member) for member in Y_train]
dim = len(X[0])

In [10]:
np.std(Y_train)

0.9993992608297877

# Model and Guide Setup

In [11]:
global prev_mu_q

prev_mu_q = torch.zeros(dim, dtype=torch.float64)

In [12]:
torch.zeros(10)

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [13]:
def model(data):
    # define the hyperparameters that control the Beta prior
    mu0 = torch.zeros(dim, dtype=torch.float64)
    b = pyro.param("b", torch.ones(dim, dtype=torch.float64), constraint=constraints.positive)
    beta = pyro.param("beta", torch.zeros(dim, dtype=torch.float64), constraint=constraints.positive)
    std = pyro.param("sigma", torch.tensor([]), constraint=constraints.positive)
    
    pyro.sample("b", dist.Gamma(torch.ones(dim, dtype=torch.float64), torch.ones(dim, dtype=torch.float64)).to_event(1))
    # sample f from the Beta prior
    pyro.sample("beta", dist.Laplace(mu0, b).to_event(1))
    pyro.sample("sigma", dist.HalfNormal(1))
    # loop over the observed data
    # subset = random.sample(data, int(len(data) / dim))
    # for i in range(len(subset)):
    # data goes list tuple tensor
          
    with pyro.plate("data", len(data), subsample_size=100) as ind:
        data = np.asarray(data)[ind]
        for i in range(len(data)):
            sampler = dist.Normal(beta.dot(data[i][0]).item(), std)
            pyro.sample("obs_{}".format(i), sampler.to_event(0), obs=data[i][0])

# NOT being used in favor of autoguide
def guide(data):
    return pyro.infer.autoguide.AutoLaplaceApproximation(model)

In [14]:
def train_SVI(D_hat, n_steps):
    losses = []
    # setup the optimizer
    adam_params = {"lr": 0.005, "betas": (0.90, 0.999)}
    optimizer = Adam(adam_params)

    # setup the inference algorithm
    guide = pyro.infer.autoguide.AutoLaplaceApproximation(model)
    svi = SVI(model, guide, optimizer, loss=Trace_ELBO())
    # do gradient steps
    for step in range(n_steps):
        loss = svi.step(D_hat)
        losses.append(loss)
        if loss < 1e-5:
            break
    breakpoint()
    mu_q = guide.state_dict()['loc'][10:20]
    return mu_q, losses

# Run VI on Data 

In [15]:
rank_proportions = []
y_hat = max(Y_train)
y_bottom = min(Y_train)
print(y_hat)
print(y_bottom)
conformal_set = []
decrease_size = 0.1
start = time.time()
print(max(Y_train))
print(min(Y_train))

all_losses = []
while y_hat >= y_bottom:
    pyro.clear_param_store()
    # Create D_hat
    D_hat = list(zip(X_train[:-1], Y_train))
    D_hat.append((X_train[-1], y_hat))
    
    # Train SVI
    mu_q, losses = train_SVI(D_hat, 200)
    all_losses.append(losses)
    prev_mu_q = mu_q
    
    # Calculate rank of y_hat
    rank = [(abs(sum(D_hat[i][0] * mu_q) - D_hat[i][1]).detach().numpy()) for i in range(len(D_hat))]
    y_hat_rank = rank[-1]
    
    # Add to conformal set if in not in bottom 10 percent of probabilities
    current_rank_proportion = np.count_nonzero(y_hat_rank > rank) / len(rank)
    rank_proportions.append(current_rank_proportion)
    if current_rank_proportion < 0.9:
        conformal_set.append(copy.deepcopy(y_hat))
        print(f"{y_hat} Added")
    else:
        print(f"{y_hat} Not added")
    y_hat -= decrease_size
conformal_set = [min(conformal_set), max(conformal_set)]
end = time.time()

tensor(2.5176, dtype=torch.float64)
tensor(-1.6510, dtype=torch.float64)
tensor(2.5176, dtype=torch.float64)
tensor(-1.6510, dtype=torch.float64)


/var/folders/1s/1mkc4t8137lbs9x8hhg9q8940000gp/T/ipykernel_80413/3203409559.py:18: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  data = np.asarray(data)[ind]
/var/folders/1s/1mkc4t8137lbs9x8hhg9q8940000gp/T/ipykernel_80413/3203409559.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  data = np.asarray(data)[ind]


ValueError: Shape mismatch inside plate('data') at site obs_0 dim -1, 100 vs 0
Trace Shapes:     
 Param Sites:     
Sample Sites:     
       b dist | 10
        value | 10
    beta dist | 10
        value | 10
   sigma dist |   
        value |   
Trace Shapes:
 Param Sites:
Sample Sites:

In [ ]:
print(f"Conformal Set: [{float(conformal_set[0])}, {float(conformal_set[1])}]")
print(f"Length: {float(conformal_set[1] - conformal_set[0])}")
print(f"Y[-1]: {Y[-1]}")
if Y[-1] >= conformal_set[0] and Y[-1] <= conformal_set[1]:
    print(f"Y[-1] is covered")
else:
    print("Y[-1] is Not covered")
print(f"Elapsed Time: {end - start}")

In [ ]:
print(all_losses[0])

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(range(len(all_losses[0])), all_losses[0])
plt.show()

In [ ]:
print(mu_q)

In [ ]:
pyro.get_param_store().keys()

In [ ]:
p_value = pyro.param("AutoLaplaceApproximation.loc/b").item()